In [1]:
from __future__ import print_function
from datetime import datetime
import pandas
import numpy

In [2]:
import sys
sys.path.append('../shifted_beta_survival/')

In [3]:
from DataHandler import DataHandler
from ShiftedBeta import ShiftedBeta
from ShiftedBetaSurvival import ShiftedBetaSurvival

In [4]:
from new_test import make_raw_article_data, print_stats

In [8]:
data = pandas.read_csv('../data/data_2yr.csv')

total_size = 1000
index = numpy.arange(data.shape[0])
numpy.random.shuffle(index)
index = index[:total_size]

data = data.iloc[index]
data.index = numpy.arange(total_size)

In [9]:
names = list(data.keys())[1:-2]

x = data[names].values
y = data.values[:, -2].astype(int)
z = data.values[:, -1].astype(int)

names = ['bias'] + names

In [10]:
sb = ShiftedBeta(verbose=True, gamma_alpha=1e1, gamma_beta=1e1)
sb.fit(y, z, x, restarts=1)

Starting Optimization with parameters:
        Samples: 1000
  gamma (alpha): 10.0
   gamma (beta): 10.0
           bias: True
          Seeds: 1

Step  |    Time    | LogLikelihood |
-----------------------------------
    1 | 0:00:10.53 |     1225.9183 |


In [11]:
print_stats(sb.alpha, sb.beta, [0], names)
for i in range(1, len(names)):
    print_stats(sb.alpha, sb.beta, [0, i], names)

bias_                | 0.85869, 40.714, 0.020655
bias_freshapp_       | 0.87873, 38.836, 0.022126
bias_mobile_         | 0.8447, 41.726, 0.019842
bias_fic_            | 0.85323, 41.668, 0.020066
bias_annual_         | 0.69648, 49.647, 0.013835
bias_monthly_        | 1.0514, 33.611, 0.030332
bias_is_contractor_  | 0.91307, 37.936, 0.023503
bias_creatives_      | 0.77034, 44.833, 0.016892
bias_trades_         | 0.90312, 38.292, 0.023041
bias_other_prof_     | 0.91039, 39.304, 0.022638


In [12]:
preds_coeff = sb.predict(x)
preds = data[['system_key']].copy()
preds['alpha'] = preds_coeff[:, 0]
preds['beta'] = preds_coeff[:, 1]
preds['churn'] = preds_coeff[:, 0] / (preds_coeff[:, 0] + preds_coeff[:, 1])

print(x[:20])
print(preds.iloc[:20])

[[1 0 0 1 0 0 0 1 0]
 [1 0 0 0 1 0 0 0 1]
 [1 0 0 0 1 0 0 0 1]
 [1 0 0 1 0 0 0 0 1]
 [1 0 0 0 1 0 1 0 0]
 [0 1 0 0 1 0 0 0 1]
 [1 0 0 0 1 0 0 0 1]
 [1 0 0 0 1 0 1 0 0]
 [1 0 0 1 0 0 0 0 1]
 [1 0 0 0 1 0 0 0 1]
 [1 0 0 0 1 0 0 1 0]
 [1 0 0 0 1 0 0 0 1]
 [1 0 0 0 1 0 1 0 0]
 [1 0 0 0 1 1 0 0 1]
 [1 0 0 1 0 0 0 0 1]
 [1 0 0 0 1 0 0 0 1]
 [1 0 0 1 0 0 1 0 0]
 [1 0 0 0 1 0 0 1 0]
 [1 0 0 0 1 0 0 0 1]
 [1 0 0 0 1 0 0 0 1]]
    system_key     alpha       beta     churn
0      1586202  0.749606  44.540352  0.016551
1      1072364  1.140694  30.950478  0.035545
2       664172  1.140694  30.950478  0.035545
3      1358151  0.755640  45.717211  0.016260
4      1460009  0.965216  35.303937  0.026613
5       564737  1.096516  33.253733  0.031922
6      1204236  1.140694  30.950478  0.035545
7      1542158  0.965216  35.303937  0.026613
8       277833  0.755640  45.717211  0.016260
9      1215776  1.140694  30.950478  0.035545
10     1134651  1.131585  30.153746  0.036170
11     1139862  1.140694  3

In [13]:
data.merge(preds, on='system_key').groupby('is_contractor').mean()[['churn']]

,churn
is_contractor,
0,0.031109
1,0.035885


In [14]:
preds

,system_key,alpha,beta,churn
0,1586202,0.749606,44.540352,0.016551
1,1072364,1.140694,30.950478,0.035545
2,664172,1.140694,30.950478,0.035545
3,1358151,0.755640,45.717211,0.016260
4,1460009,0.965216,35.303937,0.026613
5,564737,1.096516,33.253733,0.031922
6,1204236,1.140694,30.950478,0.035545
7,1542158,0.965216,35.303937,0.026613
8,277833,0.755640,45.717211,0.016260
9,1215776,1.140694,30.950478,0.035545


In [32]:
data

,system_key,freshapp,mobile,fic,annual,monthly,is_contractor,creatives,trades,other_prof,age,alive
0,359159,1,0,0,0,1,0,0,0,1,28,1
1,826716,1,0,0,0,1,0,1,0,0,2,0
2,783875,1,0,0,0,1,0,0,0,1,21,1
3,1543315,1,0,0,1,0,0,0,0,1,4,1
4,1083580,1,0,0,0,1,0,0,0,1,5,0
5,472560,1,0,0,0,1,0,0,0,1,19,0
6,1182744,1,0,0,1,0,0,0,0,1,10,1
7,701410,1,0,0,0,1,0,0,0,1,3,0
8,547422,1,0,0,0,1,0,0,0,1,28,1
9,925671,0,1,0,0,1,0,0,0,1,17,1
